In [52]:
import sem
import numpy as np
import seaborn as sns
sns.set_style('whitegrid')
import matplotlib.pyplot as plt

# Create our SEM campaign
ns_3_dir = '../../../'
script = 'aloha-throughput'
results_dir = 'aloha-results'
campaign = sem.CampaignManager.new(ns_3_dir, script, results_dir,
                                   check_repo=False, overwrite=True)

# Define the parameter space we are interested in exploring
params = {
    'nDevices': 1000,
    'simulationTime': [round(i, 4) for i in np.logspace(0.5, 4, base=60, num=100, dtype='float')],
    'radius': [7500],
    'interferenceMatrix': lambda p: ["aloha"] if p["radius"] == 0 else ["goursaud"],
    'dutyCycle': [1,0]
}
runs = 10

# Run simulations with the above parameter space
campaign.run_missing_simulations(params, runs)

Running simulations: 100%|██████████| 2000/2000 [01:08<00:00, 29.19simulation/s]


In [53]:
example_result = campaign.db.get_complete_results()[0]
display(example_result)
print(sem.utils.get_command_from_result('aloha-throughput', example_result))

{'params': {'nDevices': 1000,
  'simulationTime': 2532.9869,
  'radius': 7500,
  'interferenceMatrix': 'goursaud',
  'dutyCycle': 0,
  'RngRun': 819},
 'meta': {'id': '927ad2f2-8030-4de5-8c72-9cb07e962d35',
  'elapsed_time': 0.24874305725097656,
  'exitcode': 0},
 'output': {'stdout': '156 92\n60 33\n98 59\n143 93\n126 75\n418 177\n',
  'durations.txt': '112895 205311 369663 657407 1478655 2629631 ',
  'stderr': ''}}

python3 waf --run "aloha-throughput --nDevices=1000 --simulationTime=2532.9869 --radius=7500 --interferenceMatrix=goursaud --dutyCycle=0 --RngRun=819"


In [54]:
@sem.utils.output_labels(['G', 'S'])
def get_throughput(result):
    """
    Extract the probability of success from the simulation output
    """
    durations = [float(i)/1e6 for i in result['output']['durations.txt'].split()]
    sent = 0
    g = 0
    s = 0
    sf = 7
    for sf_performance in result['output']['stdout'].splitlines():
        sent, received = [float(i) for i in sf_performance.split()]
        current_g = sent * durations[sf-7] / result['params']['simulationTime']
        g += current_g
        if sent == 0:
            s += 0
        else:
            s += current_g * received / sent
        sf += 1
    return [g, s]

In [55]:
results = campaign.get_results_as_dataframe(get_throughput,
                                            params=params)

In [56]:
results = results.join(results.groupby(list(params.keys()))['G'].mean(), on=list(params.keys()), rsuffix='_avg')

In [57]:
sns.lineplot(data=results,
            x='G_avg', 
            y='S',
            hue='dutyCycle')
plt.ylim([0, 0.4])
plt.xlim([0, 20])
plt.show()

ValueError: Could not interpret value `DutyCycle` for parameter `hue`